# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/SRPO',
 '/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B',
 '/openbmb/VoxCPM-0.5B',
 '/ibm-granite/granite-docling-258M',
 '/google/vaultgemma-1b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/IndexTeam/IndexTTS-2-Demo',
 '/spaces/abdul9999/NoWatermark',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/InternRobotics/OmniWorld',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/SRPO
Upd

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/SRPO\nUpdated\n6 days ago\n•\n7k\n•\n862\nAlibaba-NLP/Tongyi-DeepResearch-30B-A3B\nUpdated\n4 days ago\n•\n6.26k\n•\n479\nopenbmb/VoxCPM-0.5B\nUpdated\n2 days ago\n•\n2.19k\n•\n463\nibm-granite/granite-docling-258M\nUpdated\n3 days ago\n•\n16.4k\n•\n428\ngoogle/vaultgemma-1b\nUpdated\n9 days ago\n•\n3.25k\n•\n355\nBrowse 1M+ models\nSpaces\nRunning\n14k\n14k\nDeepSite v2\n🐳\nGenera

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.**  
Hugging Face is the collaborative hub for the machine learning community, where individuals and organizations come together to create, share, and innovate in AI. 

### What We Do
- **Models**: Explore and collaborate on over **1 million models** ranging from natural language processing to computer vision and beyond.
- **Datasets**: Access **250k+ datasets** to enhance and train your models.
- **Spaces**: Develop applications in an interactive environment with over **400k applications** hosted on our platform.
  
### Our Community
Hugging Face is home to an **active community** of over 60,000 AI and ML enthusiasts. Collaborate with researchers, students, and engineers from renowned organizations like:
- **Microsoft**
- **Google**
- **Amazon**
- **Meta**
- **Intel**

With continuous contributions from our users, the community thrives on mutual support and knowledge sharing.

### Company Culture
At Hugging Face, we believe in the power of open-source collaboration. We foster an environment that emphasizes:
- **Innovation**
- **Inclusivity**
- **Transparency**

We encourage our team members to explore new ideas and push the boundaries of AI technology while maintaining a supportive and friendly atmosphere.

### Career Opportunities
Join our team and become part of a mission-driven organization! We are always looking for talented individuals who want to make a difference in the AI space. Positions are available across various departments, including:
- Data Science
- Software Engineering
- Community Engagement

**Explore opportunities [here](https://huggingface.co/jobs)** and help shape the future of AI with us!

### Enterprise Solutions
Hugging Face offers tailored solutions for organizations to leverage AI securely and efficiently. Our enterprise offerings include:
- Advanced security features 
- Access controls
- Dedicated support for smooth implementation

Start building the AI solutions of tomorrow with Hugging Face!

---

### Connect with Us
Stay updated with our latest developments and join the conversation. Follow us on:
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

For more information, visit our website: [Hugging Face](https://huggingface.co)

---

Join us in building the future of AI together!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face

### "The AI community building the future."

At Hugging Face, we are dedicated to advancing the field of Artificial Intelligence and Machine Learning. Our platform is the epicenter for collaboration, where users come together to explore, create, and innovate with hundreds of thousands of models and datasets.

---

### What We Offer

- **Models**: Access to over 1 million AI models that span a variety of tasks from natural language processing to computer vision.
  
- **Datasets**: Explore and utilize more than 250,000 datasets, making it easier to train and evaluate your models.

- **Spaces**: Create and share applications within an interactive environment that supports real-time collaboration with community members.

- **Enterprise Solutions**: Robust compute solutions designed for teams, featuring enterprise-grade security and dedicated support.

---

### Our Community

More than 50,000 organizations trust Hugging Face for their AI needs, including industry leaders like Google, Microsoft, Amazon, and Meta. Our community is vibrant, with continuous contributions from researchers, developers, and enthusiasts who frequently collaborate on projects and share their findings.

---

### Company Culture

At Hugging Face, we believe in open-source collaboration and the power of community. Our culture promotes creativity, inclusivity, and innovation. We value continuous learning and provide ample opportunities for employees to grow their skills in a supportive environment.

### Join Us

As we expand, we are always on the lookout for talented individuals to join our team. Check out our careers page to find exciting job opportunities where you can contribute to shaping the future of AI.

---

### Why Choose Hugging Face?

- **Collaboration**: Leverage the power of a community-driven platform to enhance your projects and ideas.
  
- **Innovation**: Stay at the forefront of machine learning with access to cutting-edge tools and resources.

- **Support**: With priority support and a wealth of documentation, you can get the assistance you need when you need it.

---

Join us today and be part of our journey to build the future of artificial intelligence!

*For more information, visit [Hugging Face Official Website](https://huggingface.co).*

---

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

At Hugging Face, we are redefining the machine learning landscape. Our platform serves as a collaborative hub where AI enthusiasts, researchers, and developers come together to explore, innovate, and push the boundaries of artificial intelligence. We provide access to over one million models, datasets, and applications, making it easier than ever to harness the power of AI.

## Our Mission
- **Community-Driven Collaboration**: We believe that the best innovations come from collective effort. Our platform encourages users to share, collaborate, and learn from one another.
- **Open Source Excellence**: We are committed to building foundational machine learning tools in an open-source manner, fostering transparency and collaboration in the AI community.

## What We Offer
- **Models & Datasets**: Access over 1 million AI models and 250,000 datasets for various machine learning tasks. Whether it’s text, images, videos, or audio — everything is at your fingertips.
- **Hugging Spaces**: Create, share, and run AI applications seamlessly. With 400,000+ applications available, you can bring your ideas to life.
- **Enterprise Solutions**: With options for teams and businesses ranging from $20/user/month, we offer enterprise-grade support, enhanced security, and dedicated resources to help organizations accelerate their AI journey.

## Our Customers
More than **50,000 organizations** utilize Hugging Face, including tech giants like Google, Amazon, Microsoft, and AI companies like Grammarly and Meta. We take pride in being a reliable partner for enterprises looking to leverage AI technologies. 

| **Notable Customers** | **Models Available** | **Followers** |
|-----------------------|----------------------|----------------|
| Google                | 1,050                | 30.1k          |
| Amazon                | 20                   | 3.44k          |
| Microsoft             | 424                  | 15.5k          |
| Grammarly             | 11                   | 180            |

## Company Culture
At Hugging Face, we emphasize inclusion, innovation, and empowerment. Our culture reflects a strong commitment to collaboration, learning, and growth. Team members are encouraged to contribute to open-source projects, engage with the community, and pursue their passions. We believe that a diverse and inclusive environment is vital for creative solutions and fostering innovative ideas.

### Careers at Hugging Face
We are always on the lookout for passionate individuals to join our team! Positions span a range of skills from engineering to community management. Potential candidates can expect meaningful work that contributes to the broader AI community, opportunities for professional development, and a supportive and vibrant workplace.

### Join Us!
Whether you're a prospective customer, investor, or talent seeking your next opportunity, we invite you to discover how Hugging Face is making a difference in artificial intelligence.

✨ **Explore AI Apps, Browse Models, and Join Our Community Today!**

For more information, visit [huggingface.co](https://huggingface.co).

--- 
*Stay connected with us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface).*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>